In [26]:
from azureml.core import Workspace, Experiment
from azureml.data.data_reference import DataReference
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

In [18]:
# workspace
ws = Workspace.create(
   name = 'hal',
   subscription_id = '91d27443-f037-45d9-bb0c-428256992df6',
   resource_group = 'Robots',
   location = 'West US 2',
   exist_ok = True)

In [19]:
# data
datastore = ws.datastores['tacosandburritos']
datareference = DataReference(
    datastore=datastore,
    data_reference_name="tacosandburritos",
    path_on_datastore=".")

# compute target
compute = ws.compute_targets['gandalf']

## Degine Data Processing Step

In [21]:
# -z https://aiadvocate.blob.core.windows.net/public/tacodata.zip -t train.txt
food_dataset = PipelineData(
    "training_set",
    datastore=datastore)

processStep = PythonScriptStep(
    script_name="data.py",
    arguments=["-b", datareference, "-z", "https://aiadvocate.blob.core.windows.net/public/tacodata.zip", "-t", food_dataset],
    inputs=[datareference],
    outputs=[food_dataset],
    compute_target=compute,
    source_directory="."
)

## Test Run

In [27]:
pipeline1 = Pipeline(workspace=ws, steps=[processStep])

In [28]:
# Submit the pipeline to be run
pipeline_run1 = Experiment(ws, 'burritos_tacos').submit(pipeline1)
pipeline_run1.wait_for_completion()

Created step data.py [891fff58][28d12a48-7941-4d99-8f78-427615c8dc45], (This step will run and generate new outputs)
Created data reference tacosandburritos for StepId [87bea556][7a055284-e57c-4f6b-b1bd-cecddb85757f], (Consumers of this data will generate new runs.)
Submitted pipeline run: 30846f4b-ebf9-4b69-864b-3a9e1f1e460f
PipelineRunId: 30846f4b-ebf9-4b69-864b-3a9e1f1e460f
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/91d27443-f037-45d9-bb0c-428256992df6/resourceGroups/Robots/providers/Microsoft.MachineLearningServices/workspaces/hal/experiments/burritos_tacos/runs/30846f4b-ebf9-4b69-864b-3a9e1f1e460f
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 0d6f4e74-7516-4d24-b979-ea482596a0e1
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/91d27443-f037-45d9-bb0c-428256992df6/resourceGroups/Robots/providers/Microsoft.MachineLearningServices/workspaces/hal/experiments/burritos_tacos/runs/0d6f4e74-7516-4d24-b979-ea482596a0e1
StepR

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


  Stored in directory: /root/.cache/pip/wheels/0d/2e/1c/c638b7589610d8b9358a6e5eb008edacb8b3e9b6d1edc9479f
  Stored in directory: /root/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
  Stored in directory: /root/.cache/pip/wheels/c9/de/a4/a91eec4eea652104d8c81b633f32ead5eb57d1b294eab24167
  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built json-logging-py liac-arff dill pathspec pycparser
#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_f958126b6a770f1cee9c48d3d87bd04a
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container b9b20bdb631a
 ---> f4021c25ac38
Step 9/15 : ENV PATH /azureml-envs/azureml_f958126b6a770f1cee9c48d3d87bd04a/bin:$PATH
 ---> Running in a0339c9f9ab2
Removing intermediate containe

ActivityFailedException: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "No module named 'wget'",
        "details": [],
        "debugInfo": {
            "type": "ModuleNotFoundError",
            "message": "No module named 'wget'",
            "stackTrace": "  File \"azureml-setup/context_manager_injector.py\", line 95, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_f958126b6a770f1cee9c48d3d87bd04a/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_f958126b6a770f1cee9c48d3d87bd04a/lib/python3.6/runpy.py\", line 96, in _run_module_code\n    mod_name, mod_spec, pkg_name, script_name)\n  File \"/azureml-envs/azureml_f958126b6a770f1cee9c48d3d87bd04a/lib/python3.6/runpy.py\", line 85, in _run_code\n    exec(code, run_globals)\n  File \"data.py\", line 4, in <module>\n    import wget\n"
        }
    }
}